# Table of Contents

### Contents:
- [Imports, reading in data, initial examination](#Initial-Steps)
- [Taking care of Null values](#Null-Cleaning)
- [Ensuring correct dataypes](#Datatypes)
- [Renaming Columns](#Make-the-columns-more-descriptive)
- [One hot encoding](#One-hot-encoding)
- [Save the clean data](#Save-the-cleaned-data)
- [Data dictionaries](#Data-dictionary-creation)

# Initial Steps

In [2]:
# imports

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

%matplotlib inline

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# reading in training data
train = pd.read_csv('../datasets/train.csv')

# look at first 5 rows of dataframe, see if any columns are obviously droppable (like 'Unnamed: 0')
train.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


In [4]:
# reading in test data
test = pd.read_csv('../datasets/test.csv')

# look at first 5 rows of dataframe, see if any columns are obviously droppable (like 'Unnamed: 0')
test.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,7,2007,WD
4,625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,...,0,185,0,NaN,NaN,NaN,0,7,2009,WD


In [5]:
# this isn't that useful when you have 81 columns, but does list what I usually like to check for
def initial_EDA(df):
    # print out the data types to make sure the numeric columns truly are numeric
    print(f'Do any columns contain null values?:\n{df.isnull().sum()}')
    # check the data types
    print(f'\nThe data types are:\n{df.dtypes}')
    # print the column names to make sure they are aptly named
    print(f'\nThe column names are:\n{df.columns}')
    # print the shape to see if it makes sense
    print(f'\nThe shape of your dataframe is:\n{df.shape}')
    # look at the descriptive stats for numerical columns in the data to get an initial feel for it
    print(f'\nDescriptive statistics:\n{df.describe()}')

# Null Cleaning

In [6]:
# function for Checking for Null Values 
# and seeing what percentage of the dataset the nulls are, to decide if safe to drop that column or row

def null_cleaning(df):
    null_cols = []
    problem_cols = []
    
    # looping through the columns
    for i in range(df.shape[1]):
        # counting the number of null vals in the column
        num_nulls = df.isnull().sum()[i]
        # if there are more than 0 null values, add the column to our list, and see what % of the data it is
        if num_nulls != 0:
            col_name = df.columns[i]
            percent_of_data = round(num_nulls/df.shape[0], 3)
            null_cols.append([col_name, num_nulls, percent_of_data])
            # if this column is over 20% nulls, then mark it as a problem column
            if percent_of_data >= 0.2:
                problem_cols.append(col_name)
                
    # what happens if we drop the problem columns?
    df_no_prob_cols = df.drop(columns = problem_cols)
    prob_cols_percent = df_no_prob_cols.shape[1] / df.shape[1]
    
    #what happens is we drop the problem rows too?
    df_no_nulls = df_no_prob_cols.dropna()
    prob_rows_percent = df_no_nulls.shape[0]/df_no_prob_cols.shape[0]
    
    print("The problem columns are: ", problem_cols)
    print("After dropping the problem columns, you are left with ", prob_cols_percent, "% of your columns.")
    print("After dropping the problem columns, and then dropping all rows containing nulls,\n you are left with ", 
          prob_rows_percent, "% of your rows.")
    return null_cols


In [7]:
null_cleaning(train)

The problem columns are:  ['Alley', 'Fireplace Qu', 'Pool QC', 'Fence', 'Misc Feature']
After dropping the problem columns, you are left with  0.9382716049382716 % of your columns.
After dropping the problem columns, and then dropping all rows containing nulls,
 you are left with  0.7586543149683082 % of your rows.


[['Lot Frontage', 330, 0.161],
 ['Alley', 1911, 0.932],
 ['Mas Vnr Type', 22, 0.011],
 ['Mas Vnr Area', 22, 0.011],
 ['Bsmt Qual', 55, 0.027],
 ['Bsmt Cond', 55, 0.027],
 ['Bsmt Exposure', 58, 0.028],
 ['BsmtFin Type 1', 55, 0.027],
 ['BsmtFin SF 1', 1, 0.0],
 ['BsmtFin Type 2', 56, 0.027],
 ['BsmtFin SF 2', 1, 0.0],
 ['Bsmt Unf SF', 1, 0.0],
 ['Total Bsmt SF', 1, 0.0],
 ['Bsmt Full Bath', 2, 0.001],
 ['Bsmt Half Bath', 2, 0.001],
 ['Fireplace Qu', 1000, 0.488],
 ['Garage Type', 113, 0.055],
 ['Garage Yr Blt', 114, 0.056],
 ['Garage Finish', 114, 0.056],
 ['Garage Cars', 1, 0.0],
 ['Garage Area', 1, 0.0],
 ['Garage Qual', 114, 0.056],
 ['Garage Cond', 114, 0.056],
 ['Pool QC', 2042, 0.996],
 ['Fence', 1651, 0.805],
 ['Misc Feature', 1986, 0.968]]

<span style="color:blue">
Since we would still have 93% of our columns, and then 75% of our rows after dropping nulls from the training dataset, it is safe to do so. However, it does seem like maybe the features we are dropping wouldn't be null for more expensive homes? Maybe should split this up later.
    
    
    Alley (Nominal): Type of alley access to property

       Grvl	Gravel
       Pave	Paved
       NA 	No alley access
       
    FireplaceQu (Ordinal): Fireplace quality

       Ex	Excellent - Exceptional Masonry Fireplace
       Gd	Good - Masonry Fireplace in main level
       TA	Average - Prefabricated Fireplace in main living area or Masonry Fireplace in basement
       Fa	Fair - Prefabricated Fireplace in basement
       Po	Poor - Ben Franklin Stove
       NA	No Fireplace
       
       
     Pool QC (Ordinal): Pool quality
		
       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       NA	No Pool
       
       
     Fence (Ordinal): Fence quality
		
       GdPrv	Good Privacy
       MnPrv	Minimum Privacy
       GdWo	Good Wood
       MnWw	Minimum Wood/Wire
       NA	No Fence
       
       
	Misc Feature (Nominal): Miscellaneous feature not covered in other categories
		
       Elev	Elevator
       Gar2	2nd Garage (if not described in garage section)
       Othr	Other
       Shed	Shed (over 100 SF)
       TenC	Tennis Court
       NA	None

In [8]:
# dropping the probelmatic columns from train
train.drop(columns = ['Alley', 'Fireplace Qu', 'Pool QC', 'Fence', 'Misc Feature'], inplace = True)

In [9]:
# should also drop the same columns from test
test.drop(columns = ['Alley', 'Fireplace Qu', 'Pool QC', 'Fence', 'Misc Feature'], inplace = True)

In [10]:
# dropping the rows containing nulls now
train.dropna(inplace = True)

In [11]:
# checking the shape after all that dropping
train.shape

(1556, 76)

In [12]:
# can't drop rows from the test set willy nilly because we need to make 878 predictions

# instead, replace the nulls with the mean for the numeric columns
# and replace the nans in object columns with the empty string


def replace_nans(df):
    # replacing nans in numeric columns with the mean
    numeric = df.select_dtypes(exclude=['object'])
    for col in numeric.columns:
        m = df[col].mean()
        df[col].fillna(m, inplace = True)
    
    #replacing nans in object columns with the empty string
    non_numeric = df.select_dtypes('object')
    for col in non_numeric.columns:
        df[col].fillna('', inplace = True)

In [13]:
replace_nans(test)

In [14]:
# checking that it worked ok 

test.isna().sum().sum()

0

# Dropping any other unwanted columns

In [17]:
# PID is just identification numbers and have no real measure of the quality of the house
# they are numeric and could really exaggerate certain data points in a way we don't want them to
# they may introduce irrelevant patterns

train.drop(columns = ['PID'], inplace = True)
test.drop(columns = ['PID'], inplace = True)

# Datatypes

In [18]:
train.dtypes   # too many columns for this to be useful!

Id                int64
MS SubClass       int64
MS Zoning        object
Lot Frontage    float64
Lot Area          int64
                 ...   
Misc Val          int64
Mo Sold           int64
Yr Sold           int64
Sale Type        object
SalePrice         int64
Length: 75, dtype: object

In [19]:
# this function checks the datatype of every column that contains a number in a given row
# prints the datatype of any entry that contains a number
# can be used to check if numeric values are cast as objects
# can also be used to see if entries with numbers are cast as float/int when actually you want them to be objects (like zipcodes)

# n is the index of the row you want to check

def dtype_check(df, n):
    cols_to_examine = []
    # looping through every entry in the row
    for i in range(len(df.iloc[n])):
        col_name = df.columns[i]
        # looking at the entry as a string
        entry = str(df.iloc[n][i])
        # seeing if the entry contains any numbers
        for letter in entry:
            if letter in '0123456789':
                # if there is a number in the entry, what is the datatype of that column
                d_type = df.dtypes[i]
                cols_to_examine.append([col_name, d_type, entry])
                break   #so that it only does it once, only care if there is at least one number
    return cols_to_examine

In [20]:
# check it on a few random rows
dtype_check(train,2)

[['Id', dtype('int64'), '318'],
 ['MS SubClass', dtype('int64'), '60'],
 ['Lot Frontage', dtype('float64'), '73.0'],
 ['Lot Area', dtype('int64'), '9802'],
 ['Bldg Type', dtype('O'), '1Fam'],
 ['House Style', dtype('O'), '2Story'],
 ['Overall Qual', dtype('int64'), '5'],
 ['Overall Cond', dtype('int64'), '5'],
 ['Year Built', dtype('int64'), '2006'],
 ['Year Remod/Add', dtype('int64'), '2007'],
 ['Mas Vnr Area', dtype('float64'), '0.0'],
 ['BsmtFin SF 1', dtype('float64'), '0.0'],
 ['BsmtFin SF 2', dtype('float64'), '0.0'],
 ['Bsmt Unf SF', dtype('float64'), '384.0'],
 ['Total Bsmt SF', dtype('float64'), '384.0'],
 ['1st Flr SF', dtype('int64'), '744'],
 ['2nd Flr SF', dtype('int64'), '700'],
 ['Low Qual Fin SF', dtype('int64'), '0'],
 ['Gr Liv Area', dtype('int64'), '1444'],
 ['Bsmt Full Bath', dtype('float64'), '0.0'],
 ['Bsmt Half Bath', dtype('float64'), '0.0'],
 ['Full Bath', dtype('int64'), '2'],
 ['Half Bath', dtype('int64'), '1'],
 ['Bedroom AbvGr', dtype('int64'), '3'],
 ['Kit

<span style='color:blue'>
    Honestly the datatypes all look good to me! I feel like I got lucky? Seems to good to be true...

# Make the columns more descriptive

Make names that look 'Like This' instead look 'like_this'. Only problem is that for names that look 'LikeThis' get changed to 'likethis'. But it's only a couple columns so if I really wanted to I could do that manually.

In [21]:
def change_names(df):
    name_dict = {}
    for name in df.columns:
        new_name = name.lower().replace(' ','_')
        name_dict[name] = new_name
    return df.rename(columns = name_dict)

In [22]:
train = change_names(train)

In [23]:
test = change_names(test)

# One hot encoding

In [24]:
# adding a sale price to the test data
# sale price is nonzero for everything in train
# so we'll keep track of what is train and what is test this way while we merge and unmerge

sale_price = np.zeros_like(test['id'])
test['saleprice'] = sale_price

In [25]:
# check that train and test have the same number of columns
test.shape[1] == train.shape[1]

True

In [26]:
# merge the data frames, just stack the rows
train_and_test = pd.concat([train, test])

In [27]:
# make sure the merge went ok

print(train_and_test.shape[1] == test.shape[1])
print(train_and_test.shape[0] == test.shape[0]+train.shape[0])

True
True


In [28]:
# this function will turn any object columns of a dataframe into dummies
# it returns the dummied dataframe

def dummy_the_objects(df):
    non_numeric = df.select_dtypes('object')
    return pd.get_dummies(df, columns = non_numeric.columns, drop_first = True)

In [29]:
# dummying the merged dataframe 

train_and_test_d = dummy_the_objects(train_and_test)

In [30]:
# making sure we actually added more coumns

train_and_test_d.shape

(2434, 246)

In [31]:
# filter to get just the dummied train data 

train_d = train_and_test_d[(train_and_test_d['saleprice']) != 0]

In [32]:
# filter to get just the dummied test data

test_d = train_and_test_d[(train_and_test_d['saleprice']) == 0]

In [33]:
train_d.shape

(1556, 246)

In [34]:
# drop the sale price column of zeros from the test set 

test_d.drop(columns = 'saleprice', inplace = True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [35]:
test_d.shape

(878, 245)

In [36]:
# now we have train and test with the same dummies

# Data dictionary creation

# Save the cleaned data

In [37]:
# saving the cleaned data with no dummies

train.to_csv('../datasets/train_clean_no_dum.csv', index = False)
test.to_csv('../datasets/test_clean_no_dum.csv', index = False)

In [38]:
# saving the cleaned data with dummies 

train_d.to_csv('../datasets/train_clean_dum.csv', index = False)
test_d.to_csv('../datasets/test_clean_dum.csv', index = False)


### Contents:
- [Imports, reading in data, initial examination](#Initial-Steps)
- [Taking care of Null values](#Null-Cleaning)
- [Ensuring correct dataypes](#Datatypes)
- [Renaming Columns](#Make-the-columns-more-descriptive)
- [One hot encoding](#One-hot-encoding)
- [Save the clean data](#Save-the-cleaned-data)
- [Back to top](#Table-of-Contents)